In [1]:
#connects jupyter notebook to spark engine
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.4.1-bin-hadoop3'

In [2]:
#time module required to capture time and measure performance
import time 
start_time = time.time()

In [3]:
#Initiate spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Covid Time Series Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
import pyspark.sql.functions as F

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [6]:
#Reading CSV file into the code 
df = spark.read.options(header=True, inferSchema = True).csv('time_series_covid19_confirmed_global.csv')
df.show()

+--------------------+-------------------+---------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

Processing Pipeline

In [7]:
#Rename columns
df = df.withColumnRenamed("Country/Region", "Country")
df = df.withColumnRenamed("Province/State", "State")

In [8]:
#exclude outliers
excluded_countries = ["Diamond Princess", "MS Zaandam", "Summer Olympics 2020", "Winter Olympics 2022"]
df = df.filter(~F.col("Country").isin(excluded_countries))

In [9]:
#Pivot Date and Daily test case count to make data easy to work with
id_vars = ["State", "Country", "Lat", "Long"]
value_vars = [f'"{c}",`{c}`' for c in df.columns if c not in id_vars ]

stack_expr = F.expr(f"stack({len(value_vars)}, {', '.join(value_vars)}) as (date, value)")
df_reshaped = df.select(id_vars + [stack_expr])

In [10]:
#convert date column from string to date data type
df_reshaped = df_reshaped.withColumn("date", F.to_date("date","M/d/yy"))

df_reshaped.show()

+-----+-----------+--------+---------+----------+-----+
|State|    Country|     Lat|     Long|      date|value|
+-----+-----------+--------+---------+----------+-----+
| null|Afghanistan|33.93911|67.709953|2020-01-22|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-23|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-24|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-25|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-26|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-27|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-28|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-29|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-30|    0|
| null|Afghanistan|33.93911|67.709953|2020-01-31|    0|
| null|Afghanistan|33.93911|67.709953|2020-02-01|    0|
| null|Afghanistan|33.93911|67.709953|2020-02-02|    0|
| null|Afghanistan|33.93911|67.709953|2020-02-03|    0|
| null|Afghanistan|33.93911|67.709953|2020-02-04|    0|
| null|Afghanistan|33.93911|67.709953|2020-02-05

Cells Specific To Question1

In [11]:
#Query to first Question
df_monthly_avg = ( 
    df_reshaped
    .groupBy("Country", F.month("date").alias("month"), F.year("date").alias("year"))
    .agg({'value':'avg'})
    .withColumnRenamed('avg(value)',"avg_daily_confirmed")
)

In [12]:
df_final1 = df_monthly_avg.orderBy(['Country','year','month'])

df_final1.show()

+-----------+-----+----+-------------------+
|    Country|month|year|avg_daily_confirmed|
+-----------+-----+----+-------------------+
|Afghanistan|    1|2020|                0.0|
|Afghanistan|    2|2020| 1.0344827586206897|
|Afghanistan|    3|2020| 36.806451612903224|
|Afghanistan|    4|2020|              838.4|
|Afghanistan|    5|2020| 7184.5161290322585|
|Afghanistan|    6|2020| 25056.166666666668|
|Afghanistan|    7|2020|  34819.74193548387|
|Afghanistan|    8|2020|  37583.83870967742|
|Afghanistan|    9|2020|  38880.03333333333|
|Afghanistan|   10|2020| 40215.967741935485|
|Afghanistan|   11|2020| 43462.166666666664|
|Afghanistan|   12|2020|  49662.93548387097|
|Afghanistan|    1|2021|  53911.58064516129|
|Afghanistan|    2|2021|  55467.92857142857|
|Afghanistan|    3|2021|  56034.45161290323|
|Afghanistan|    4|2021| 57775.333333333336|
|Afghanistan|    5|2021|  64256.45161290323|
|Afghanistan|    6|2021|  94957.23333333334|
|Afghanistan|    7|2021| 136817.67741935485|
|Afghanist

In [13]:
end_time = time.time()

In [14]:
total_time = end_time-start_time

In [15]:
print(total_time)
print(start_time)
print(end_time)

48.74311637878418
1700429038.3589716
1700429087.102088


In [16]:
df_final1.write.csv("Answer to Question 1", header=True, mode="overwrite") 